In [12]:

pip install accelerate
pip install -i https://pypi.org/simple/ bitsandbytes --upgrade

SyntaxError: invalid syntax (1534730467.py, line 1)

In [13]:
import transformers
import textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import os
import sys
from typing import List
# import fire
import torch
# from datasets import load_dataset
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pylab import rcParams

%matplotlib inline
sns.set(rc={'figure.figsize':(10, 7)})
sns.set(rc={'figure.dpi':100})
sns.set(style='white', palette='muted', font_scale=1.2)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cpu'

In [14]:
BASE_MODEL = "NousResearch/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
def create_prompt(text, tokenizer):
    messages = [
        {
            "role": "system",
            "content": "You are a friendly chatbot who always responds as superhuman intelligence AI",
        },
        {"role": "user", "content": text},
    ]
    return tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

In [ ]:
response = []

In [ ]:
prompts = ["You are the best primary school teacher based in rural India. You want to explain the concepts of cube and cube roots to the students in the most simplest way possible. For this you use examples and stories to make things easier to understand. Prepare a chapter as instructed for the NCERT chapter Cube and Cube roots covering each and every detail.",
           ]

In [ ]:
text = "skfsdkfhksdbf"

In [ ]:
prompt = [create_prompt(text, tokenizer)]

In [ ]:
model_inputs = tokenizer(prompt, return_tensors='pt').to(DEVICE)

In [ ]:
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=2000,
    temperature = 0.1,
    # do_sample=True,
    # top_p=0.92,
    # top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Output:
----------------------------------------------------------------------------------------------------
[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI
<</SYS>>

You are the best primary school teacher based in rural India. You want to explain the concepts of cube and cube roots to the students in the most simplest way possible. For this you use examples and stories to make things easier to understand. Prepare a chapter as instructed for the NCERT chapter Cube and Cube roots covering each and every detail. [/INST]  Title: The Amazing World of Cubes and Cube Roots!

Introduction:
Hello there, young explorers! Today, we're going to embark on a thrilling adventure through the world of cubes and cube roots! Are you ready to discover the magic of these fascinating mathematical concepts? 😃

Chapter 1: What is a Cube?

* Definition: A cube is a three-dimensional shape with all sides of equal length.
* Examples: Think of a cube as a box, a ball,

In [ ]:
# Your existing code

ground_truth_responses = [
    "The chapter covers the concepts of cube and cube roots in a simple and engaging manner.",
]

generated_response = tokenizer.decode(sample_output[0], skip_special_tokens=True)

# Ground truth comparison
if generated_response in ground_truth_responses:
    print("Generated response matches ground truth.")
else:
    print("Generated response does not match ground truth.")

# Metric evaluation (example using BLEU score)
from nltk.translate.bleu_score import sentence_bleu

# Calculate BLEU score between generated response and ground truth responses
bleu_scores = [sentence_bleu([truth.split()], generated_response.split()) for truth in ground_truth_responses]
average_bleu = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU score:", average_bleu)

# Human evaluation (manual inspection)
print("Generated response:", generated_response)
print("Ground truth responses:", ground_truth_responses)

# Domain-specific constraints (e.g., checking for specific keywords)
if "cube" in generated_response and "root" in generated_response:
    print("Generated response meets domain-specific constraints.")
else:
    print("Generated response does not meet domain-specific constraints.")


In [ ]:
response.append(tokenizer.decode(sample_output[0], skip_special_tokens=True))

In [ ]:
# AT THE END
import json
data_dict = dict(zip(prompts, response))
# Convert dictionary to JSON string
file_path = "data.json"

# Save JSON object to file
with open(file_path, "w") as file:
    json.dump(data_dict, file)
# json_string = json.dumps(key_dict)
# # Print the JSON string
# print(json_string)